In [1]:
import os
import mlflow
import tempfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [5]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
cuda = torch.cuda.is_available()
seed = 1
log_interval = 100

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
cuda

True

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, 
        transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
        batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, 
        transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [9]:
iter(train_loader).next()[0].shape

torch.Size([64, 1, 28, 28])

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

    def log_weights(self, step):
        writer.add_histogram('weights/conv1/weight', model.conv1.weight.data, step)
        writer.add_histogram('weights/conv1/bias', model.conv1.bias.data, step)
        writer.add_histogram('weights/conv2/weight', model.conv2.weight.data, step)
        writer.add_histogram('weights/conv2/bias', model.conv2.bias.data, step)
        writer.add_histogram('weights/fc1/weight', model.fc1.weight.data, step)
        writer.add_histogram('weights/fc1/bias', model.fc1.bias.data, step)
        writer.add_histogram('weights/fc2/weight', model.fc2.weight.data, step)
        writer.add_histogram('weights/fc2/bias', model.fc2.bias.data, step)

model = Net()
if cuda:
    model.cuda()

In [13]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [17]:
writer = None # Will be used to write TensorBoard events

def log_scalar(name, value, step):
    """Log a scalar value to both MLflow and TensorBoard"""
    writer.add_scalar(name, value, step)
    mlflow.log_metric(name, value)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            step = epoch * len(train_loader) + batch_idx
            log_scalar('train_loss', loss.data.item(), step)
            model.log_weights(step)

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').data.item() # sum up batch loss
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
    step = (epoch + 1) * len(train_loader)
    log_scalar('test_loss', test_loss, step)
    log_scalar('test_accuracy', test_accuracy, step)



In [36]:
args = {
    'batch_size': batch_size,
    'test_batch_size': test_batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'cuda': cuda,
    'seed': seed,
    'log_interval': log_interval}


with mlflow.start_run():
    # Log our parameters into mlflow
    for key, value in args.items():
        mlflow.log_param(key, value)

    # Create a SummaryWriter to write TensorBoard events locally
    output_dir = tempfile.mkdtemp()
    writer = SummaryWriter(output_dir)
    print("Writing TensorBoard events locally to %s\n" % output_dir)

    # Perform the training
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)

    # Upload the TensorBoard event logs as a run artifact
    print("Uploading TensorBoard events as a run artifact...")
    mlflow.log_artifacts(output_dir, artifact_path="events")
    print("\nLaunch TensorBoard with:\n\ntensorboard --logdir=%s" %
        os.path.join(mlflow.get_artifact_uri(), "events"))

    output_dir = tempfile.mkdtemp()
    torch.save(model, os.path.join(output_dir, 'model.pth'))
    mlflow.log_artifacts(output_dir, artifact_path="models")
    mlflow.log_artifacts('./notebooks', artifact_path="notebooks")

Writing TensorBoard events locally to /tmp/tmput70y1ap

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.130347
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.052099
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.141832
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.097194
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.082489
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.147342
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.022296
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.089414
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.057757
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.072311

Test set: Average loss: 4.6732, Accuracy: 9846/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.270308
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.094780
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.027853
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.235094
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.121951
Train Epoch: 2 [32000/60000 (53%)]	Loss: 2.002873
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.102097
Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.21

/home/adam/anaconda3/envs/mlflow-aca447097abf603ea9b9a7196081ebc9950a2bd8/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
